## Day 23 Lecture 1 Assignment

In this assignment, we will explore feature selection and dimensionality reduction techniques. We will use both the FIFA ratings dataset and the Chicago traffic crashes dataset.

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [0]:
crash_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/traffic_crashes_chicago.csv')
soccer_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/fifa_ratings.csv')

In [0]:
soccer_data.head()

,ID,Name,Overall,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle
0,158023,L. Messi,94,84,95,70,90,86,97,93,94,87,96,91,86,91,95,95,85,68,72,59,94,48,22,94,94,75,96,33,28,26
1,20801,Cristiano Ronaldo,94,84,94,89,81,87,88,81,76,77,94,89,91,87,96,70,95,95,88,79,93,63,29,95,82,85,95,28,31,23
2,190871,Neymar Jr,92,79,87,62,84,84,96,88,87,78,95,94,90,96,94,84,80,61,81,49,82,56,36,89,87,81,94,27,24,33
3,192985,K. De Bruyne,91,93,82,55,92,82,86,85,83,91,91,78,76,79,91,77,91,63,90,75,91,76,61,87,94,79,88,68,58,51
4,183277,E. Hazard,91,81,84,61,89,80,95,83,79,83,94,94,88,95,90,94,82,56,83,66,80,54,41,87,89,86,91,34,27,22


We will begin with the Chicago traffic crashes dataset, focusing on removing columns with significant missing data.

Remove all columns with more than 5% missing data from the dataframe. (The *missingness summary* function we wrote a few exercises ago will speed this process up significantly.) Print out the columns that were removed, and the proportion of missing data for each column.

In [0]:
# answer goes here
def missingness_summary(df, print_log, sort):
  freqlist = df.apply(lambda x: x.isnull()/df.shape[0])
 
  if print_log == True:
    if sort == 'descending':
      print(freqlist.sum().sort_values(axis = 0, ascending=False))
    else:
      print(freqlist.sum().sort_values(axis = 0, ascending=True))

missingness_summary(crash_data, print_log=True, sort='ascending')
  




RD_NO                            0.000000
STREET_NO                        0.000000
SEC_CONTRIBUTORY_CAUSE           0.000000
DATE_POLICE_NOTIFIED             0.000000
DAMAGE                           0.000000
CRASH_TYPE                       0.000000
ROAD_DEFECT                      0.000000
ROADWAY_SURFACE_COND             0.000000
ALIGNMENT                        0.000000
PRIM_CONTRIBUTORY_CAUSE          0.000000
TRAFFICWAY_TYPE                  0.000000
FIRST_CRASH_TYPE                 0.000000
LIGHTING_CONDITION               0.000000
WEATHER_CONDITION                0.000000
CRASH_DATE                       0.000000
POSTED_SPEED_LIMIT               0.000000
DEVICE_CONDITION                 0.000000
TRAFFIC_CONTROL_DEVICE           0.000000
STREET_NAME                      0.000003
STREET_DIRECTION                 0.000005
BEAT_OF_OCCURRENCE               0.000011
NUM_UNITS                        0.003755
INJURIES_UNKNOWN                 0.005776
INJURIES_REPORTED_NOT_EVIDENT    0

In [0]:
dfdrop = crash_data.copy()
dfdrop.drop(['LANE_CNT', 'HIT_AND_RUN_I','INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I','STATEMENTS_TAKEN_I','PHOTOS_TAKEN_I','WORK_ZONE_I','WORK_ZONE_TYPE','DOORING_I','WORKERS_PRESENT_I'], axis=1)

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN
0,JC334993,7/4/2019 22:33,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,"OVER $1,500",7/4/2019 23:05,FOLLOWING TOO CLOSELY,NOT APPLICABLE,300,N,LAKE SHORE DR SB,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JC370822,7/30/2019 10:22,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,"OVER $1,500",7/30/2019 10:25,FAILING TO YIELD RIGHT-OF-WAY,IMPROPER TURNING/NO SIGNAL,8201,S,DR MARTIN LUTHER KING JR DR,631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JC387098,8/10/2019 17:00,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,"$501 - $1,500",8/10/2019 17:35,EQUIPMENT - VEHICLE CONDITION,NOT APPLICABLE,6747,S,CREGIER AVE,332.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JC395195,8/16/2019 16:53,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,"$501 - $1,500",8/16/2019 16:53,UNABLE TO DETERMINE,NOT APPLICABLE,554,N,FRANKLIN ST,1831.0,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,JC396604,8/17/2019 16:04,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,PARKING LOT,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,NO INJURY / DRIVE AWAY,"$501 - $1,500",8/17/2019 18:30,UNABLE TO DETERMINE,UNABLE TO DETERMINE,3700,N,WESTERN AVE,1921.0,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372580,JC378354,8/4/2019 15:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,UNKNOWN,NaN,INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",8/4/2019 16:27,FAILING TO REDUCE SPEED TO AVOID CRASH,NOT APPLICABLE,4520,S,PULASKI RD,815.0,2.0,NONINCAPACITATING INJURY,1.0,0.0,0.0,1.0,0.0,1.0,0.0
372581,JC406679,8/25/2019 13:07,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",8/25/2019 13:08,FOLLOWING TOO CLOSELY,FOLLOWING TOO CLOSELY,5958,W,LAWRENCE AVE,1622.0,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,0.0
372582,JC406342,8/25/2019 6:00,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NaN,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",8/25/2019 7:00,PHYSICAL CONDITION OF DRIVER,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,10400,S,MICHIGAN AVE,512.0,2.0,NONINCAPACITATING INJURY,1.0,0.0,0.0,1.0,0.0,0.0,0.0
372583,JC386106,8/9/2019 21:24,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,"RUT, HOLES",ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",8/9/2019 21:24,UNABLE TO DETERMINE,UNABLE TO DETERMINE,3806,N,KEDZIE AVE,1733.0,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,3.0,0.0


Next, we will shift our focus to the FIFA ratings dataset and explore univariate feature selection techniques. We will treat "Overall" as the response and the other ratings as features.

Using the correlations between the response and features, identify the 5 features with the greatest univariate correlation to the response.

In [0]:
# answer goes here
sdcorr = soccer_data.corr()
sdcorroverall = soccer_data.corr().iloc[1:,1]
print(sdcorr.head(10))
print(sdcorroverall.head(10))

np.abs(sdcorroverall).sort_values(ascending=False).head(7)




                       ID   Overall  ...  StandingTackle  SlidingTackle
ID               1.000000 -0.426009  ...       -0.168401      -0.138797
Overall         -0.426009  1.000000  ...        0.264082       0.224365
Crossing        -0.277281  0.496603  ...        0.085429       0.078963
Finishing       -0.169946  0.373079  ...       -0.558565      -0.585841
HeadingAccuracy -0.288233  0.466908  ...        0.257420       0.234933
ShortPassing    -0.315288  0.722720  ...        0.235083       0.203009
Volleys         -0.277298  0.452090  ...       -0.402222      -0.430056
Dribbling       -0.149494  0.516363  ...       -0.244737      -0.258046
Curve           -0.307825  0.503054  ...       -0.128246      -0.147523
FKAccuracy      -0.327434  0.455875  ...       -0.056328      -0.081250

[10 rows x 31 columns]
Overall            1.000000
Crossing           0.496603
Finishing          0.373079
HeadingAccuracy    0.466908
ShortPassing       0.722720
Volleys            0.452090
Dribbling       

Overall         1.000000
Reactions       0.847739
Composure       0.801749
ShortPassing    0.722720
BallControl     0.717933
LongPassing     0.585104
ShotPower       0.562960
Name: Overall, dtype: float64

Use sklearn's "SelectKBest" function to select the top 5 features using two different scoring metrics: f_regression and mutual_info_regression. Print out the top 5 columns that are selected by both. How do they compare to the ones selected by  univariate correlation?

In [0]:
# answer goes here
Y = soccer_data['Overall']
X = soccer_data.drop(['Overall', 'Name'], axis=1)
from sklearn.feature_selection import SelectKBest, f_classif, f_regression, mutual_info_regression

k = 5
kbest = SelectKBest(k=k, score_func=f_classif)
X_best_feat = kbest.fit_transform(X,Y)

X_best_feat = pd.DataFrame(X_best_feat, columns=X.columns[kbest.get_support()])

X_best_feat.columns





Index(['ShortPassing', 'LongPassing', 'BallControl', 'Reactions', 'Composure'], dtype='object')

In [0]:
kbest = SelectKBest(k=k, score_func=f_regression)
X_best_feat = kbest.fit_transform(X,Y)

X_best_feat = pd.DataFrame(X_best_feat, columns=X.columns[kbest.get_support()])

X_best_feat.columns

Index(['ShortPassing', 'LongPassing', 'BallControl', 'Reactions', 'Composure'], dtype='object')

In [0]:
kbest = SelectKBest(k=k, score_func=mutual_info_regression)
X_best_feat = kbest.fit_transform(X,Y)

X_best_feat = pd.DataFrame(X_best_feat, columns=X.columns[kbest.get_support()])

X_best_feat.columns

Index(['ShortPassing', 'Dribbling', 'BallControl', 'Reactions', 'Composure'], dtype='object')

Shifting our focus from feature selection to dimensionality reduction, perform PCA on the ratings provided, excluding "Overall". Then, answer the following questions:

- What percentage of the total variance is capture by the first component? What about the first two, or first three?
- Looking at the components themselves, how would you interpret the first two components in plain English?

In [0]:
# answer goes here
small_data = df_features.loc[:, ['quant_log_conn', 'scale_log_consume_per_connection']].copy()
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
small_data_pca = pca.fit_transform(small_data)
print(pca.mean_)
print(pca.components_)
print(pca.explained_variance_)

xdata = small_data['quant_log_conn']
ydata = small_data['scale_log_consume_per_connection']

def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(xdata, ydata, alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal')
plt.xlabel('quant_log_conn')
plt.ylabel('scale_log_consume_per_connection')
plt.title('2 Components')
plt.show()


